# MSADS509 M4 UE Wang 

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [3]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            -- your query here
                            SELECT text, party
                            FROM conventions
                            ''')

punctuation = set(punctuation) # speeds up comparison
sw = stopwords.words("english")
whitespace_pattern = re.compile(r"\s+")

def remove_stop(tokens) :
    
    return [t for t in tokens if t.lower() not in sw]

def remove_punctuation(text, punct_set=punctuation) : 
    
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    
    return re.split(whitespace_pattern, text)

def prepare(text, pipeline) : 
    
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

for row in query_results :
    text,party = row
    prep = prepare(text, pipeline=pipeline)
    speeches = " ".join(prep)
    convention_data.append([speeches, party])

Let's look at some random entries and see if they look right. 

In [4]:
random.choices(convention_data,k=10)

[['joe biden understanding average american experiencing think he’s',
  'Democratic'],
 ['god bless united states', 'Republican'],
 ['someone navigated thorny issues policy castigating alone also encouraging people better think',
  'Democratic'],
 ['inaudible 021441', 'Democratic'],
 ['thank', 'Republican'],
 ['kicking doctors social media yet promoting chinese state funded propaganda major tech platforms election critical since 1860 man named lincoln elected preserve union disintegration election important lifetime important since preservation republic 1865 reelecting trump ensure kids raised love country respect founding fathers taught hate ashamed build monuments heroes burn cities we’ll country rises higher heights dreams big thinks big achieves impossible country values remarkable journey complexities past clearly communicates next generation grateful angry live united states',
  'Republican'],
 ['immediately upon taking office president trump changed things change threatened esta

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [5]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [6]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
        
    ret_dict = dict()
    
    for word in fw:
        if word in text.split():
            ret_dict[word] = True
    
    return(ret_dict)

In [7]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [8]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [9]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [10]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [11]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._


Out of the 25 informative features, 23 strongly associated with Republican, while 2 are associated with Democrats. The high ratio values underscore the significance of these features.

Republican-leaning terms like 'China,' 'enforcement,' 'enemy,' and 'defense' dominate the list, aligning with typical Republican political viewpoints. On the Democratic side, 'votes' and 'climate' represent concerns in line with their political perspectives.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [12]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [13]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [14]:
tweet_data = []

for row in results :
    candidate, party, tweet = row
    decode = tweet.decode('utf-8')
    prep = prepare(text=decode, pipeline=pipeline)
    prepped_text = " ".join(prep)
    tweet_data.append([prepped_text, party])

In [15]:
tweet_data[0:1]

[['brooks joins alabama delegation voting flawed funding bill httptco3cwjiwysnq',
  'Republican']]

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [16]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [17]:
random.choices(tweet_data,k=10)

[['perrynelson thanks', 'Democratic'],
 ['great ag news today → restoring trade access china us beef producers incredible step forward httpstcorpl2dw3dec',
  'Republican'],
 ['icymi bluevirginia jennifer discusses need replace politicians blame mentally ill acts gun violence httpstcoilynevbpqw',
  'Democratic'],
 ['two days womensmarch realdonaldtrumps choice bring back globalgag rule reminds us must fight protectourcare',
  'Democratic'],
 ['today 130pm president barack obama honor recipients 2010 medal freedom ceremony httpfbmevke2veh0',
  'Democratic'],
 ['asked usda inspector general fong recent survey found scientists usda worry could face punishment researching issues relative climate change pollinator health antimicrobial resistance mepolitics watch➡️ httpstcoacisxw8s8a',
  'Democratic'],
 ['happy presidents day realdonaldtrump presidentsday2018 httpstcol8xenamvap',
  'Republican'],
 ['foxx votes require collection unpaid taxes federal employees httptinyurlcom3ajgnh6',
  'Republ

In [18]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr c

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [19]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [20]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3806, 'Democratic': 608}),
             'Democratic': defaultdict(int,
                         {'Republican': 4662, 'Democratic': 926})})

### Reflections

_Write a little about what you see in the results_ 

For the Republican class, 3806 instances were True Positives (TP) and correctly predicted as Republican, while 608 instances were False Negatives (FN) and mispredicted as Democratic. In the Democratic class, 4662 instances were False Positives (FP) and incorrectly predicted as Republican, and 926 instances were True Negatives (TN) and correctly predicted as Democratic. This implies an accuracy of (TP + TN) / (TP + FP + FN + TN), which evaluates to 47.3%. The observed accuracy is lower than my anticipated expectations.